In [6]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
from dotenv import load_dotenv

from collab_data.file_utils import expand_path, get_project_root
from collab_data.gcs_utils import GCSClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Grab API keys

In [7]:
# Load environment variables from .env file
load_dotenv()

gerbils_key = os.environ.get("COLLAB_GERBILS_KEY")
data_key = os.environ.get("COLLAB_DATA_KEY")

print(f"Gerbils key: {gerbils_key}")
print(f"Data key: {data_key}")

Gerbils key: /workspace/api-keys/collab-gerbils-444419-2c6aa9a47bb5.json
Data key: /workspace/api-keys/collab-data-463313-c340ad86b28e.json


### Grab current project data

In [8]:
CURRENT_PROJECT = "COLLAB_DATA"
PROJECT_KEY = Path(os.environ.get(f"{CURRENT_PROJECT}_KEY"))
PROJECT_ID = "-".join(PROJECT_KEY.stem.split("-")[:-1])

# Connect to GCS
gcs_client = GCSClient(
    project_id=PROJECT_ID,
    credentials_path=expand_path(PROJECT_KEY.as_posix(), get_project_root()),
)

2025-07-23 14:35:45.524 | INFO     | collab_data.gcs_utils:__init__:34 - Using credentials from /workspace/api-keys/collab-data-463313-c340ad86b28e.json
2025-07-23 14:35:45.600 | INFO     | collab_data.gcs_utils:__init__:40 - Using project collab-data-463313


### Find current folders

In [9]:
all_buckets = gcs_client.list_buckets()
print(f"Available buckets: {all_buckets}")

Available buckets: ['fieldwork_curated', 'fieldwork_processed', 'test-bucket-collab-data_344a2055-565d-4023-9666-4c2fd5c225b1']


### Start with the curated

In [11]:
curated_data_dir = 'fieldwork_curated'

# Find all files in the curated fieldwork
curated_files = gcs_client.glob(f"{curated_data_dir}/*")

curated_fn = curated_files[0]


# file_path = gcs_client.glob(f"{all_buckets[-1]}/field/*final*")[0]

# # Create a local path for downloading
# local_data_path = Path("./data") / Path(file_path).name

# # Download the file
# gcs_client.gcs.get_file(
#     rpath=file_path,
#     lpath=local_data_path.as_posix(),
# )

In [14]:
Path(curated_fn).name

'2024_05_18-session_0001'

In [ ]:
datadir = "/workspace/fieldwork-data/birds"
local_data_path = Path()